# 一般的な画像データの処理
今までは, PyTorch が提供するベンチマークデータセットを取り扱っていた.  
今回は普通の画像データ(拡張子は `.png`)を用いた一般的な画像処理のやり方を学習する.  
まずは必要なライブラリをインポートする.

In [ ]:
# 必要なライブラリのインポート
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from tqdm import tqdm

ノートブック全体で使う関数を定義する.

In [ ]:
def outputs_setter(model: nn.Module, dataloader: DataLoader) -> torch.Tensor:
    """
    モデルの出力を取得する関数

    Parameters
    ----------
    model: nn.Module
        学習済みモデル
    inputs: torch.Tensor
        入力データ

    Returns
    ----------
    outputs: torch.Tensor
        モデルの出力データ
    """
    # モデルを評価モードに設定
    model.eval()

    # 勾配計算を無効化
    with torch.no_grad():
        # モデルの出力を取得
        images, _ = next(iter(dataloader))
        outputs = model(images)

    return outputs


def plot_data(train_loss_list: list) -> None:
    """
    学習過程の損失をプロットする関数

    Parameters
    ----------
    train_loss_list: list
        学習過程の損失を格納したリスト

    Returns
    ----------
    None
    """
    # 横軸の設定
    x = [i + 1 for i in range(len(train_loss_list))]

    # 出力画像の設定
    plt.figure(figsize=(18, 12), tight_layout=True)
    plt.title("Training Loss over Epochs", size=15, color="red")
    plt.grid()
    plt.xlabel("Epoch")
    plt.ylabel("Loss")

    # 学習過程の損失をプロット
    plt.plot(x, train_loss_list)

    # グラフの表示
    plt.show()

画像データを適切なディレクトリに配置し, `ImageFolder` を使うことで簡単にデータセットに変換できる.

In [ ]:
# 乱数シードの設定
seed = 42
torch.manual_seed(seed)

# 画像の前処理の定義
data_transform = transforms.Compose(
    [transforms.Resize((256, 256)), transforms.ToTensor()]
)

# 画像データセットの読み込み
batch_size = 64
dataset = ImageFolder("./", transform=data_transform)
dataloader = DataLoader(dataset, batch_size=batch_size)

# データの確認
for images, _ in dataloader:
    plt.imshow(images[0].permute(1, 2, 0))
    plt.show()
    break

## 畳み込みオートエンコーダを使った画像の再構成
**オートエンコーダ** とは, 入力次元を圧縮して **潜在表現** を学習し, その潜在表現から元のデータを復元するニューラルネットワークの1種である.  
このオートエンコーダのアーキテクチャに畳み込みを使ったものが **畳み込みオートエンコーダ** である.

In [ ]:
class CNNEncoder(nn.Module):
    def __init__(self):
        # 親クラスのコンストラクタを呼び出す
        super(CNNEncoder, self).__init__()

        # 畳み込み層の定義
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, stride=2, padding=1)

        # 活性化関数の定義
        self.relu = nn.ReLU()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        順伝播を行う関数

        Parameters
        ----------
        x: torch.Tensor
            入力データ

        Returns
        ----------
        z: torch.Tensor
            潜在表現データ
        """
        # 1層目の計算
        h = self.relu(self.conv1(x))

        # 2層目の計算
        h = self.relu(self.conv2(h))

        # 3層目の計算
        h = self.relu(self.conv3(h))

        # 4層目の計算
        z = self.relu(self.conv4(h))

        return z


class CNNDecoder(nn.Module):
    def __init__(self):
        # 親クラスのコンストラクタを呼び出す
        super(CNNDecoder, self).__init__()

        # 転置畳み込み層の定義
        self.deconv1 = nn.ConvTranspose2d(
            512, 256, kernel_size=3, stride=2, padding=1, output_padding=1
        )
        self.deconv2 = nn.ConvTranspose2d(
            256, 128, kernel_size=3, stride=2, padding=1, output_padding=1
        )
        self.deconv3 = nn.ConvTranspose2d(
            128, 64, kernel_size=3, stride=2, padding=1, output_padding=1
        )
        self.deconv4 = nn.ConvTranspose2d(
            64, 3, kernel_size=3, stride=2, padding=1, output_padding=1
        )

        # 活性化関数の定義
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, z: torch.Tensor) -> torch.Tensor:
        """
        順伝播を行う関数

        Parameters
        ----------
        z: torch.Tensor
            潜在表現データ

        Returns
        ----------
        out: torch.Tensor
            出力データ
        """
        # 1層目の計算
        h = self.relu(self.deconv1(z))

        # 2層目の計算
        h = self.relu(self.deconv2(h))

        # 3層目の計算
        h = self.relu(self.deconv3(h))

        # 4層目の計算
        out = self.sigmoid(self.deconv4(h))

        return out


class CNNAutoencoder(nn.Module):
    def __init__(self):
        # 親クラスのコンストラクタを呼び出す
        super(CNNAutoencoder, self).__init__()

        # エンコーダとデコーダの定義
        self.encoder = CNNEncoder()
        self.decoder = CNNDecoder()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        順伝播を行う関数

        Parameters
        ----------
        x: torch.Tensor
            入力データ

        Returns
        ----------
        out: torch.Tensor
            出力データ
        """
        # エンコーダの計算
        z = self.encoder(x)

        # デコーダの計算
        out = self.decoder(z)

        return out

モデルのアーキテクチャを確認する.

In [ ]:
# モデルのインスタンスを作成
model = CNNAutoencoder()
print(model)

学習のハイパーパラメータ等を設定する.

In [ ]:
# 最適化手法の設定
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# 損失関数の設定
criterion = nn.MSELoss()

# エポック数の設定
num_epochs = 200

# 学習履歴を保存するリストの初期化
train_loss_list = []

データを学習する.

In [ ]:
# 再構成画像の表示回数の設定
show_num = 10
interval = num_epochs // show_num

# 学習ループの実行
with tqdm(range(num_epochs)) as pbar_epoch:
    # 最初にモデルの出力を取得
    outputs = outputs_setter(model, dataloader)

    # 表示する画像の初期設定
    _, axes = plt.subplots(2 + show_num, 8, figsize=(12, 2 * (2 + show_num)))
    for i in range(8):
        axes[0, i].imshow(images[i].permute(1, 2, 0))
        axes[0, i].axis("off")
        axes[1, i].imshow(outputs[i].permute(1, 2, 0))
        axes[1, i].axis("off")

    # エポックごとのループ
    for epoch in pbar_epoch:
        # エポック数の表示
        pbar_epoch.set_description(f"Epoch {epoch + 1}")

        # モデルを学習モードに設定
        model.train()

        # エポックの損失を初期化
        epoch_loss = 0.0

        # ミニバッチ学習の実行
        for images, _ in dataloader:
            # 勾配の初期化
            optimizer.zero_grad()

            # 順伝播の計算
            with torch.set_grad_enabled(True):
                outputs = model(images)
                loss = criterion(outputs, images)

                # 逆伝播の計算とパラメータの更新
                loss.backward()
                optimizer.step()

            # エポックの損失を更新
            epoch_loss += loss.item() * images.size(0)

        # エポックの平均損失を計算
        epoch_loss /= len(dataloader.dataset)
        train_loss_list.append(epoch_loss)

        # 進捗バーに損失を表示
        pbar_epoch.set_postfix(loss=epoch_loss)

        # 再構成画像を表示
        if (epoch + 1) % interval == 0:
            # モデルの出力を取得
            outputs = outputs_setter(model, dataloader)

            # 元画像と再構成画像をプロット
            for i in range(8):
                axes[(epoch + 1) // interval + 1, i].imshow(
                    outputs[i].permute(1, 2, 0)
                )
                axes[(epoch + 1) // interval + 1, i].axis("off")

    # 画像を表示
    plt.show()

エポックを重ねるごとに再構成の精度が上がっていることが確認できる.  
ただし, 60エポックくらいを過ぎたところから, もう大分元画像に近いものが生成できているため, それ以降の違いはあまり無いようにも受け止められる.  
ロスの確認を行う.

In [ ]:
# 学習ロス曲線を描画
plot_data(train_loss_list)

大体ロスも収束し, このオートエンコーダの生成が上手くいっている事がわかる.

## 潜在表現空間の可視化
オートエンコーダの中で, エンコーダの出力する潜在表現をベクトルとして可視化することを考える.

In [ ]:
# 潜在表現と画像データの取得
latents = []
images = []
for imgs, _ in dataloader:
    z = model.encoder(imgs)
    latents.append(z.view(z.size(0), -1).numpy())
    images.append(imgs)

# 潜在表現をまとめる
latents = np.concatenate(latents, axis=0)

# 画像データをまとめる
images = torch.cat(images, dim=0)

上記で取得したエンコーダの潜在表現ベクトルは高次元すぎるため, **PCA(主成分分析)** によってノイズになる次元を削除し, **t-SNE** で非線形圧縮をして3次元に変換する.

In [ ]:
# PCAによる次元削減
pca = PCA(n_components=50)
latents_pca = pca.fit_transform(latents)

# t-SNEによる次元削減
tsne = TSNE(n_components=3, random_state=seed)
latents_tsne = tsne.fit_transform(latents_pca)
print(latents_tsne.shape)